![Обработка данных](https://vibrantwebtech.com/wp-content/uploads/2018/07/Marketing-Intelligence-Data-Entry-and-Data-Processing-Services.jpg)

# Прогнозирование стоимости автомобиля по его характеристикам
Часть 2
__________________________________________________________________________________________________________________

**Этот ноутбук включает:**
> - анализ структуры и формы собранных с внешнего сайта [auto.ru](https://auto.ru/)  данных;
> - предобработку собранных с внешнего сайта [auto.ru](https://auto.ru/)  данных;
> - сохранение обработанных данных в итоговом файле для использования их в моделировании;

# Библиотеки

In [1]:
import re #бибилиотека для работы с регулярными выражениями (используется для вытаскивания нужного текста/цифр)

#общие библиотеки
import pandas as pd
import numpy as np

# Блок предобработки данных

In [2]:
#загрузим данные из буферного файла, в котром хранится очередная порция собранных с сайта авто.ру данных
data_auto = pd.read_csv('C:/Users/Алексей Третьяков/Desktop/ML_projects/FierceDra-X.github.io/module_5/data_auto_f.csv')

#посмотрим на структуру данных, чтобы понимать, как их приводить к формату данных, соответствующих тестовому сету
data_auto.info()
data_auto.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   bodytype       74 non-null     object
 1   brand          74 non-null     object
 2   mileage        74 non-null     object
 3   color          74 non-null     object
 4   engine         74 non-null     object
 5   complectation  60 non-null     object
 6   tax            74 non-null     object
 7   transmission   74 non-null     object
 8   drive          74 non-null     object
 9   wheel          74 non-null     object
 10  owners         74 non-null     object
 11  customs        74 non-null     object
 12  pts            74 non-null     object
 13  year           74 non-null     object
 14  price          74 non-null     object
dtypes: object(15)
memory usage: 8.9+ KB


,bodytype,brand,mileage,color,engine,complectation,tax,transmission,drive,wheel,owners,customs,pts,year,price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Кузовседан,Audi A4 35 TFSI V (B9) Рестайлинг,Пробег25 км,Цветбелый,Двигатель2.0 л / 150 л.с. / Бензин,Комплектация22 опции,Налог5 250 ₽ / год,Коробкароботизированная,Приводпередний,РульЛевый,Владельцы1 владелец,ТаможняРастаможен,ПТСОригинал,год выпуска2021,3 299 000 ₽Смотреть статистику цен
2,Кузовседан,Audi A4 35 TFSI V (B9) Рестайлинг,Пробег20 000 км,Цветчёрный,Двигатель2.0 л / 150 л.с. / Бензин,NaN,Налог3 750 ₽ / год,Коробкароботизированная,Приводпередний,РульЛевый,Владельцы1 владелец,ТаможняРастаможен,ПТСОригинал,год выпуска2020,3 160 000 ₽Смотреть статистику цен
3,Кузовседан,Audi A4 IV (B8) Рестайлинг,Пробег202 600 км,Цветкоричневый,Двигатель1.8 л / 170 л.с. / Бензин,NaN,Налог8 500 ₽ / год,Коробкавариатор,Приводпередний,РульЛевый,Владельцы2 владельца,ТаможняРастаможен,ПТСОригинал,год выпуска2014,880 000 ₽Смотреть статистику цен
4,Кузовуниверсал 5 дв.,Audi A4 V (B9),Пробег90 000 км,Цветчёрный,Двигатель2.0 л / 150 л.с. / Дизель,Комплектация1 опция,Налог5 250 ₽ / год,Коробкароботизированная,Приводпередний,РульЛевый,Владельцы1 владелец,ТаможняРастаможен,ПТСОригинал,год выпуска2017,2 181 000 ₽Хорошая ценаСмотреть статистику цен


In [3]:
#удалим ненужную первую строку
data_auto.drop(index = [0], inplace = True)

#будем удалаять попавшиеся объявления по новым автомобилям, они идентифицируются по значению "Предложение от дилера"
#также удалим все строки с отутствием цены автомобиля, так как цена нам необходима как таргет в обучении
data_auto.drop(data_auto[data_auto['brand'] == 'Предложение от дилера'].index, inplace=True)
data_auto.drop(data_auto[data_auto.price.isna()].index, inplace=True)

In [4]:
#удалим слово Кузов в столбце bodytype, оставив только символы после пятого символа
data_auto['bodytype'] = data_auto['bodytype'].apply(lambda x: x[5:])

In [5]:
#из столбца brand вытащим наименование модели авто, создав под нее новый столбец
#вытащим модель, разбив строку на список, взяв второй элемент в списке
#и очистим от пробелов по краям, сделав все буквы заглавными
def model (model):
    return model.split()[1].strip().upper()
data_auto['model_name'] = data_auto['brand'].apply(model)

In [6]:
#точно также вытащим бренд автомобиля, разбив данные на список и взяв первый элемент из списка
def brand (brand):
    return brand.split()[0].upper()
data_auto['brand'] = data_auto['brand'].apply(brand)

In [7]:
#в колонке mileage с помощью библиотеки re фильтруем цифровые символы и преобразуем их в int тип данных
def mileage(mileage):
    return int(re.sub("[^0-9]","",mileage))
data_auto['mileage'] = data_auto['mileage'].apply(mileage)

In [8]:
#в колонке color убираем первые 4 буквы Цвет
#в колонке year убираем первые 11 букв и преобразовываем текст в int тип данных
data_auto['color'] = data_auto['color'].apply(lambda x: x[4:])
data_auto['year'] = data_auto['year'].apply(lambda x: int(x[11:]))

In [9]:
#из колонки engine сформируем три колонки:
#1. Объем двигателя, взяв цифры и точки, разбив их на список и взяв первый элемент из списка, преобразовав его в тип float;
#2. Мощность двигателя, взяв цифры, разбив их на список и взяв второй элемент из списка, преобразовав его в тип int;
#3. Тип топлива, разбив их на список и взяв третий элемент из списка;
data_auto['engineDisplacement'] = data_auto['engine'].apply(lambda x: float(re.sub("[^0-9\.]","",x.split('/')[0][9:12])))
data_auto['enginePower'] = data_auto['engine'].apply(lambda x: int(re.sub("[^0-9]","",x.split('/')[1])))
data_auto['fuelType'] = data_auto['engine'].apply(lambda x: x.split('/')[2])
data_auto.drop('engine', axis=1, inplace=True)

In [10]:
#часть объема двигателя, которое получилось больше 10, приеобразуем в типовой объем 2 литра
def eDispl(eDispl):
    if eDispl > 10:
        return 2.0
    else:
        return eDispl
data_auto['engineDisplacement'] = data_auto['engineDisplacement'].apply(eDispl)

In [11]:
#там, где есть не пустые значения комплектации, вытащим количество комплектаций и преобразуем его в тип int
#остальные ячейки заполним нулями, предположив, что комплектация у этих авто скудная
def compl(compl):
    try:
        if compl.find('опц'):
            return int(re.sub("[^0-9]","",compl[13:15]))
        else:
            return 0
    except:
        return 0

data_auto['complectation'] = data_auto['complectation'].apply(compl)

In [12]:
#из колонки tax вытащим фильтрацией цифры и преобразуем их в тип int
def tax(tax):
    try:
        return int(re.sub("[^0-9]","",tax))
    except:
        return 0

data_auto['tax'] = data_auto['tax'].apply(tax)

In [13]:
#в колонках ниже убираем первое ненужное слово
data_auto['transmission'] = data_auto['transmission'].apply(lambda x: x[7:])
data_auto['drive'] = data_auto['drive'].apply(lambda x: x[6:])
data_auto['wheel'] = data_auto['wheel'].apply(lambda x: x[4:])

#в колонке customs есть nan, заменим их типовым значением ТаможняРастаможен
#далее уберем семь первых символа Таможня
data_auto['customs'].fillna('ТаможняРастаможен', inplace=True)
data_auto['customs'] = data_auto['customs'].apply(lambda x: x[7:])

#в колонке pts есть nan, заменим их типовым значением ПТСОригинал
#далее уберем три первых символа ПТС
data_auto['pts'].fillna('ПТСОригинал', inplace=True)
data_auto['pts'] = data_auto['pts'].apply(lambda x: x[3:])

In [14]:
#из колонки owners вытащим фильтрацией цифры и преобразуем их в тип int
def own(owners):
    try:
        return int(re.sub("[^0-9]","",owners))
    except:
        return 1

data_auto['owners'] = data_auto['owners'].apply(own)

In [15]:
#из колонки price вытащим фильтрацией цифры и символ рубля
#разделим строку на список и возьмем первый элемент с цифрами, преобразовав их в тип int
def price(price):
    pr = re.sub("[^0-9₽]","", price).split('₽')
    return int(pr[0])

data_auto['price'] = data_auto['price'].apply(price)

In [16]:
#добавим очищенные данные конкатенацией к имеющимся в основном файле data_auto_processed_FINAL_3 очищенным данным
data_final = pd.read_csv('C:/Users/Алексей Третьяков/Desktop/ML_projects/FierceDra-X.github.io/module_5/data_auto_processed_FINAL_3.csv')
data = pd.concat([data_final,data_auto])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86362 entries, 0 to 74
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bodytype            86362 non-null  object 
 1   brand               86362 non-null  object 
 2   mileage             86362 non-null  int64  
 3   color               86362 non-null  object 
 4   complectation       86362 non-null  int64  
 5   tax                 86362 non-null  int64  
 6   transmission        86362 non-null  object 
 7   drive               86362 non-null  object 
 8   wheel               86362 non-null  object 
 9   owners              86362 non-null  int64  
 10  customs             86361 non-null  object 
 11  pts                 86362 non-null  object 
 12  year                86362 non-null  int64  
 13  price               86362 non-null  int64  
 14  model_name          86362 non-null  object 
 15  engineDisplacement  86362 non-null  float64
 16  engineP

In [17]:
#запишем дополненный датасет с очищенными данными в основной файл data_auto_processed_FINAL_3.csv
#файл обновится
data.to_csv('C:/Users/Алексей Третьяков/Desktop/ML_projects/FierceDra-X.github.io/module_5/data_auto_processed_FINAL_3.csv', index=False)

```
основной файл с очищенными данными data_auto_processed_FINAL_3 далее в третьей части проекта будет использован в моделировании и прогнозировании стоимости б-у автомобилей
  ```